# OSIC Pulmonary Fibrosis Progression

**In this [new competition](https://www.kaggle.com/c/osic-pulmonary-fibrosis-progression/overview/description) we are asked to predict lung function based on the output of a spirometer (FVC)- a device that measures the volumne of air inhaled/exhaled, metadata, and CT images in the form of DICOM files. The competition is particularly interesting in the way the data is structured. We are given a baseline chest CT image/FVC score and patient data at `Week = 0`. A single patient has multiple visits over the course of 1-2 years, at which time their final FVC is measured. We need to predict the FVC of each patient *every possible week* to prevent potential data leakage.**

# DICOM Overview

**DICOM stands for Digital Imaging and COmmunications in Medicine and it is the standard format that allows information to be shared between different medical imaging equipment like X-Ray, MRI, and CT. DICOM files have a `.dcm` extension that allows one to store meta-data about the image inside the file, like patient ID or patient gender**

**In general, CT images use 12-bit DICOM files with pixel values ranging from -1024 to 3071, whereas a normal greyscale image has pixels values between 0 and 225. The pixel ranges in DICOM files are such that they align with the '[Hounsfield Scale](https://radiopaedia.org/articles/hounsfield-unit?lang=us)' which arbitrarily defined such that the radiodensity of water at STP = 0 and the radiodensity of air at STP = -1000 on the HU scale** 

**References: most of this notebook is inspired by [Jeremy Howard](https://www.kaggle.com/jhoward) and his below notebooks on `fastai2` and its implementation for DICOM images. He is a huge inspiration of mine and I recommend you study not only the below notebooks, but all of his notebooks:**
> https://www.kaggle.com/jhoward/don-t-see-like-a-radiologist-fastai/data  
> https://www.kaggle.com/jhoward/cleaning-the-data-for-rapid-prototyping-fastai  
> https://www.kaggle.com/jhoward/some-dicom-gotchas-to-be-aware-of-fastai

**I also did not see this notebook until now, but [this notebook](https://www.kaggle.com/nxrprime/fibrosis-eda-fast-ai) by [Trigram](https://www.kaggle.com/nxrprime) covers the same contents of this kernel and much more. It is excellent work and I highly suggest you read it**

In [ ]:
#the Python basics
import numpy as np, matplotlib.pyplot as plt, pandas as pd, seaborn as sns
import math, os, time, cv2

#update pip
!pip install --upgrade --quiet pip

#get fastai2
print('Installing fastai2...')
!pip install --quiet fastai2
print('Installation complete')

# FastAI

**[FastAI](https://github.com/fastai/fastai) is a library that sits on top of PyTorch that allows for fast and simple neural network experiments. In fact, you can train a MNIST model with ResNet18 in just a few lines of code like so:**

In [ ]:
from fastai.vision import *

In [ ]:
#train ResNet on MNIST sample in 4 lines of code
path = untar_data(URLs.MNIST_SAMPLE)
data = ImageDataBunch.from_folder(path)
learn = cnn_learner(data, models.resnet50, metrics=accuracy)
#learn.fit(1)

**Now that is impressive! With only 4 lines of code, we were able to train a [ResNet](https://arxiv.org/abs/1512.03385) with 50 hidden layers or ~ 23 million trainable parameters. But FastAI is not just for training, it also has a host of other submodules, most importantly the `fastai2.medical.imaging` submodule that we can use for this OSIC competition that makes working with DICOM files much easier**

# FastAI Medical Imaging

**Radiologists use something called [windowing](https://radiopaedia.org/articles/windowing-ct?lang=us) to manipulate the greyscale components of an image, highlighting certain structures. They do this because a healthy human can only see 100 different color shades, and even fewer when the observed colors are presented through a computer screen, so raw DICOM images look washed out without any sort of windowing. Let's see what an image looks like before windowing and after:**

**Note that `fastai2` is rapidly developing so you may need to tweak some code based on current documentation**

In [ ]:
from fastai2.basics import *
from fastai2.vision.all import *
from fastai2.medical.imaging import *

In [ ]:
#choose different windowing options
scales = False, dicom_windows.lungs
titles = 'Raw','Windowed - Lung'

#compare different windowing options
files = glob.glob('../input/osic-pulmonary-fibrosis-progression/train/*/*.dcm')
file = Path(np.random.choice(files))
dcm = file.dcmread()

for s,a,t in zip(scales, subplots(1,2,imsize=7.3)[1].flat, titles):
    dcm.show(scale=s, ax=a, title=t)

**Now that is much better: after using `dicom_windows.lungs`, the image is not as blurry and we can clearly see the details of the lungs. But this is not all we can do with `fastai2.medical.imaging`. For one, it can actually tell us the percentage of pixels in the window like so:**

In [ ]:
print(file)

In [ ]:
#check percentage of pixels in window
dcm.pct_in_window(*dicom_windows.lungs)

**Now there is a follow up question: humans might need windowing to view DICOM images, but do *computers* need windowing for training? No, because a neural network accepts data in the form of floating points, which use 32 bits. If we decide to use floats, we can't use PIL or save them as JPEGs, but we can use `fastai.vision`**

**We can't just forget scaling altogether, however. It is still good practice to scale any type of input when feeding it to a machine learning model, and this is still true for images. Ideally, we want something Gaussian-ish to describe our pixel value distribution. Let's see if this is the case:** 

In [ ]:
pixel_dist = dcm.scaled_px.flatten()
fig, ax = plt.subplots(figsize = (20,7))
sns.kdeplot(np.asarray(pixel_dist), shade = True)
plt.show()

**We can see that we have a trimodal distribution here, which is not at all what we want to see. So how do we rescale it? Well, we scale the pixel values with some type of non-linear mapping that gives us an equal number of pixels in each range (think `pandas.qcut`)**

**Let's put this in a function and check the other files in our ID directory:**

In [ ]:
#create bins and plot new distribution
bins = pixel_dist.freqhist_bins(20)
print(bins)
fig, ax = plt.subplots(figsize = (20,7))
plt.hist(pixel_dist, bins=bins)
plt.show()

**We won't be able to get a perfectly uniform distribution here simple because some values occur many times and some rarely occur at all. To create a function that connects these bins with a line, we can do this:**

In [ ]:
#connect with smooth line
fig, ax = plt.subplots(figsize = (20, 7))
plt.plot(bins, torch.linspace(0,1,len(bins)))
plt.show()

**And now we just need to find a way to apply this mapping...luckily `fastai2.medical.imaging` can do this for us, in fact we didn't need to do the above binning at all, we can simply use the method `dcm.hist_scaled()`. Now when we do this, we must use a new method, `show_image(s)` because the methods you can call on the `dcm` object result in Tensors; e.g. this**

<code>
dcm_scaled = dcm.hist_scaled()
dcm_scaled.show()   
</code>
<br/>

**will result in an `AttributeError: 'Tensor' object has no attribute 'show'`. You can dig deeper into this [here](https://github.com/fastai/fastai2/blob/master/fastai2/medical/imaging.py#L179). Also note that `hist_scaled()` is the default way of displaying a DICOM image in FastAI. If we were to use `plt.imshow` with this object, we get:**

In [ ]:
#plotting image with plt.imshow()
plt.imshow(dcm.hist_scaled(), cmap=plt.cm.bone)
plt.show()

In [ ]:
#plotting image with fastai show()
dcm.show()

**Yep, those are the same! Now we can also use the `show` method in combination with the scale parameter to compare the raw, windowed, and scaled images:**

In [ ]:
#choose different windowing options
scales = False, dicom_windows.lungs, True
titles = 'Raw','Windowed - Lung', 'Scaled'

for s,a,t in zip(scales, subplots(1,3,imsize=7)[1].flat, titles):
    dcm.show(scale=s, ax=a, title=t)

**Now, you might be wondering: this is great for a single image, but how is this applied to a set of images? The way we have applied this rescaling, it will vary from image to image. So, we need to create a new rescaling map that works for every image in our dataset**

**But we should ensure our images contain only important information before doing this. One thing we can do is crop our images to just the lung area:**

# Crop To Lung Area Only

**We can use blurring to help us with this cropping, so let's first explore how to apply blurring transformations to our DICOM files**

**However, you should know that `show_image(s)` defaults to the colormap `viridis`. We can change this to the default color map that `show` uses like so:**

In [ ]:
matplotlib.rcParams['image.cmap'] = 'bone'

In [ ]:
#view different blurring effects
images = dcm.hist_scaled(), uniform_blur2d(dcm.hist_scaled(),50), gauss_blur2d(dcm.hist_scaled(),50)
show_images(images, titles=('Scaled', 'Blurred', 'Gaussian Blurred'), imsize = 7)

**Now we are ready to crop to the lung area. We can use blurring to crop out non-lung parts of the image like so:**
1. We window the image with `windowed(*dicom_windows.lungs)`
2. We blur the image with `gauss_blur`
3. We select only the bright parts of the image
4. Use results from 3) as a 'mask' to place over our image, effectively giving us the area of the photo with lung tissue only

**Let's go through this procedure manually and then use `mask_from_blur` method to demonstrate how easy this is to implement with FastAI2:**

## Manually

**I have cherry picked a perfect example of an image that needs to be drastically cropped for this part of the notebook**

In [ ]:
dcm_to_crop = Path('../input/osic-pulmonary-fibrosis-progression/train/ID00122637202216437668965/13.dcm').dcmread()

In [ ]:
#step 1 - window
windowed = dcm_to_crop.windowed(*dicom_windows.lungs)
show_image(windowed, cmap = plt.cm.bone)
plt.show()

In [ ]:
#step 2 - blur
blurred = gauss_blur2d(windowed, 10)
show_image(blurred, cmap = plt.cm.bone)
plt.show()

In [ ]:
#step 3 - select bright parts of blur
show_image(blurred > .35, cmap = plt.cm.bone)
plt.show()

**We would then overlay the mask ontop of the scaled image and resize to the mask, essentially zooming in on the important parts of the image. Let's now see how to do this with `mask_from_blur`**

## With mask_from_blur

**Even though the above sequence of steps was not too involved, we can make our lives even easier by using `mask_from_blur` combined with `mask2bbox` like so:**

**The below code is taken from [Jeremy Howard](https://www.kaggle.com/jhoward)'s notebook [here](https://www.kaggle.com/jhoward/cleaning-the-data-for-rapid-prototyping-fastai)**

In [ ]:
mask = dcm_to_crop.mask_from_blur(dicom_windows.brain)
wind = dcm_to_crop.windowed(*dicom_windows.brain)

_,ax = subplots(1,1, figsize = (7,7))
show_image(wind, ax=ax[0])
show_image(mask, alpha=0.5, ax=ax[0]);

In [ ]:
bbs = mask2bbox(mask)
lo,hi = bbs
show_image(wind[lo[0]:hi[0],lo[1]:hi[1]], figsize = (7, 7));

**Well that worked! Now we need to resize to the original size**

In [ ]:
print(f"Shape of decm: {dcm.shape}")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (12, 12))
dcm_cropped = crop_resize(to_device(wind[None]), bbs[...,None], dcm.shape[0])[0]
show_image(windowed, ax = ax[0], title = 'Uncropped')
show_image(dcm_cropped, ax = ax[1], title = 'Cropped')
plt.show()

In [ ]:
#sanity check
print(f"Shape of cropped image: {dcm_cropped.shape}")

**Since this image has a 'microscope' border, it is already cropped as tightly as it can be, so our function does not do anything. But for any images in the dataset that are not in this microscope format, it will focus in on the important part of the CT image**

# Get DICOM Metadata

**As we discussed earlier, DICOM files have internal data besides images that we can use. FastAI makes it very easy to extract this data and we can simply turn it into a DataFrame like so:** 

In [ ]:
#extract from directory and convert to dataframe
file_dir = Path('../input/osic-pulmonary-fibrosis-progression/train/ID00007637202177411956430/')
dicom_meta = pd.DataFrame.from_dicoms(file_dir.ls())

#check size and view
print(f"Extracted DICOM data is of dimension: {dicom_meta.shape}")
dicom_meta.head()

**Well that's a lot of data for a single ID number. This extra data will likely provide useful for training, so we will explore it more later**

# Better Than Scaling & Windowing?

**We can take advantage of different rainbow colormaps in `matplotlib` to visualize our image in full color. We have some options we can compare side by side**

In [ ]:
print(f"Current matplotlib version: {matplotlib.__version__}")

## Matplotlib Colormaps

In [ ]:
#visualize DICOM image with other colormaps
fig, ax = plt.subplots(4, 4, figsize = (17, 17))
dcm.show(cmap=plt.cm.gist_ncar, ax = ax[0,0], title = 'gist_ncar')
dcm.show(cmap=plt.cm.jet, ax = ax[0,1], title = 'jet')
dcm.show(cmap=plt.cm.gist_rainbow_r, ax = ax[0,2], title = 'gist_rainbow_r')
dcm.show(cmap=plt.cm.PRGn, ax = ax[0,3], title = 'PRGn')
dcm.show(cmap=plt.cm.rainbow, ax = ax[1,0], title = 'rainbow')
dcm.show(cmap=plt.cm.gnuplot2, ax = ax[1,1], title = 'gnuplot2')
dcm.show(cmap=plt.cm.rainbow, ax = ax[1,2], title = 'cube_helix')
dcm.show(cmap=plt.cm.brg, ax = ax[1,3], title = 'brg')
dcm.show(cmap=plt.cm.terrain, ax = ax[2,0], title = 'terrain')
dcm.show(cmap=plt.cm.gist_stern, ax = ax[2,1], title = 'gist_stern')
dcm.show(cmap=plt.cm.nipy_spectral, ax = ax[2,2], title = 'nipy_spectral')
dcm.show(cmap=plt.cm.viridis, ax = ax[2,3], title = 'viridis')
dcm.show(cmap=plt.cm.coolwarm, ax = ax[3,0], title = 'coolwarm')
dcm.show(cmap=plt.cm.viridis_r, ax = ax[3,1], title = 'viridis_r')
dcm.show(cmap=plt.cm.twilight, ax = ax[3,2], title = 'twilight')
dcm.show(cmap=plt.cm.twilight_shifted, ax = ax[3,3], title = 'twilight_shifted')

## Turbo

**There is also `turbo` which was added to a recent version of `matplotlib`, which you can read about [here](https://ai.googleblog.com/2019/08/turbo-improved-rainbow-colormap-for.html), but we need to download it first - I can't figure out how to access it through `matplotlib`; when I update to 3.3.0 I get new errors, so we can install copy paste it from this [GitHub repository](https://gist.github.com/FedeMiorelli/640bbc66b2038a14802729e609abfe89)**

**Turbo is most similar to jet, so we will compare the two side by side:**

In [ ]:
turbo_colormap_data = np.array(
                       [[0.18995,0.07176,0.23217],
                       [0.19483,0.08339,0.26149],
                       [0.19956,0.09498,0.29024],
                       [0.20415,0.10652,0.31844],
                       [0.20860,0.11802,0.34607],
                       [0.21291,0.12947,0.37314],
                       [0.21708,0.14087,0.39964],
                       [0.22111,0.15223,0.42558],
                       [0.22500,0.16354,0.45096],
                       [0.22875,0.17481,0.47578],
                       [0.23236,0.18603,0.50004],
                       [0.23582,0.19720,0.52373],
                       [0.23915,0.20833,0.54686],
                       [0.24234,0.21941,0.56942],
                       [0.24539,0.23044,0.59142],
                       [0.24830,0.24143,0.61286],
                       [0.25107,0.25237,0.63374],
                       [0.25369,0.26327,0.65406],
                       [0.25618,0.27412,0.67381],
                       [0.25853,0.28492,0.69300],
                       [0.26074,0.29568,0.71162],
                       [0.26280,0.30639,0.72968],
                       [0.26473,0.31706,0.74718],
                       [0.26652,0.32768,0.76412],
                       [0.26816,0.33825,0.78050],
                       [0.26967,0.34878,0.79631],
                       [0.27103,0.35926,0.81156],
                       [0.27226,0.36970,0.82624],
                       [0.27334,0.38008,0.84037],
                       [0.27429,0.39043,0.85393],
                       [0.27509,0.40072,0.86692],
                       [0.27576,0.41097,0.87936],
                       [0.27628,0.42118,0.89123],
                       [0.27667,0.43134,0.90254],
                       [0.27691,0.44145,0.91328],
                       [0.27701,0.45152,0.92347],
                       [0.27698,0.46153,0.93309],
                       [0.27680,0.47151,0.94214],
                       [0.27648,0.48144,0.95064],
                       [0.27603,0.49132,0.95857],
                       [0.27543,0.50115,0.96594],
                       [0.27469,0.51094,0.97275],
                       [0.27381,0.52069,0.97899],
                       [0.27273,0.53040,0.98461],
                       [0.27106,0.54015,0.98930],
                       [0.26878,0.54995,0.99303],
                       [0.26592,0.55979,0.99583],
                       [0.26252,0.56967,0.99773],
                       [0.25862,0.57958,0.99876],
                       [0.25425,0.58950,0.99896],
                       [0.24946,0.59943,0.99835],
                       [0.24427,0.60937,0.99697],
                       [0.23874,0.61931,0.99485],
                       [0.23288,0.62923,0.99202],
                       [0.22676,0.63913,0.98851],
                       [0.22039,0.64901,0.98436],
                       [0.21382,0.65886,0.97959],
                       [0.20708,0.66866,0.97423],
                       [0.20021,0.67842,0.96833],
                       [0.19326,0.68812,0.96190],
                       [0.18625,0.69775,0.95498],
                       [0.17923,0.70732,0.94761],
                       [0.17223,0.71680,0.93981],
                       [0.16529,0.72620,0.93161],
                       [0.15844,0.73551,0.92305],
                       [0.15173,0.74472,0.91416],
                       [0.14519,0.75381,0.90496],
                       [0.13886,0.76279,0.89550],
                       [0.13278,0.77165,0.88580],
                       [0.12698,0.78037,0.87590],
                       [0.12151,0.78896,0.86581],
                       [0.11639,0.79740,0.85559],
                       [0.11167,0.80569,0.84525],
                       [0.10738,0.81381,0.83484],
                       [0.10357,0.82177,0.82437],
                       [0.10026,0.82955,0.81389],
                       [0.09750,0.83714,0.80342],
                       [0.09532,0.84455,0.79299],
                       [0.09377,0.85175,0.78264],
                       [0.09287,0.85875,0.77240],
                       [0.09267,0.86554,0.76230],
                       [0.09320,0.87211,0.75237],
                       [0.09451,0.87844,0.74265],
                       [0.09662,0.88454,0.73316],
                       [0.09958,0.89040,0.72393],
                       [0.10342,0.89600,0.71500],
                       [0.10815,0.90142,0.70599],
                       [0.11374,0.90673,0.69651],
                       [0.12014,0.91193,0.68660],
                       [0.12733,0.91701,0.67627],
                       [0.13526,0.92197,0.66556],
                       [0.14391,0.92680,0.65448],
                       [0.15323,0.93151,0.64308],
                       [0.16319,0.93609,0.63137],
                       [0.17377,0.94053,0.61938],
                       [0.18491,0.94484,0.60713],
                       [0.19659,0.94901,0.59466],
                       [0.20877,0.95304,0.58199],
                       [0.22142,0.95692,0.56914],
                       [0.23449,0.96065,0.55614],
                       [0.24797,0.96423,0.54303],
                       [0.26180,0.96765,0.52981],
                       [0.27597,0.97092,0.51653],
                       [0.29042,0.97403,0.50321],
                       [0.30513,0.97697,0.48987],
                       [0.32006,0.97974,0.47654],
                       [0.33517,0.98234,0.46325],
                       [0.35043,0.98477,0.45002],
                       [0.36581,0.98702,0.43688],
                       [0.38127,0.98909,0.42386],
                       [0.39678,0.99098,0.41098],
                       [0.41229,0.99268,0.39826],
                       [0.42778,0.99419,0.38575],
                       [0.44321,0.99551,0.37345],
                       [0.45854,0.99663,0.36140],
                       [0.47375,0.99755,0.34963],
                       [0.48879,0.99828,0.33816],
                       [0.50362,0.99879,0.32701],
                       [0.51822,0.99910,0.31622],
                       [0.53255,0.99919,0.30581],
                       [0.54658,0.99907,0.29581],
                       [0.56026,0.99873,0.28623],
                       [0.57357,0.99817,0.27712],
                       [0.58646,0.99739,0.26849],
                       [0.59891,0.99638,0.26038],
                       [0.61088,0.99514,0.25280],
                       [0.62233,0.99366,0.24579],
                       [0.63323,0.99195,0.23937],
                       [0.64362,0.98999,0.23356],
                       [0.65394,0.98775,0.22835],
                       [0.66428,0.98524,0.22370],
                       [0.67462,0.98246,0.21960],
                       [0.68494,0.97941,0.21602],
                       [0.69525,0.97610,0.21294],
                       [0.70553,0.97255,0.21032],
                       [0.71577,0.96875,0.20815],
                       [0.72596,0.96470,0.20640],
                       [0.73610,0.96043,0.20504],
                       [0.74617,0.95593,0.20406],
                       [0.75617,0.95121,0.20343],
                       [0.76608,0.94627,0.20311],
                       [0.77591,0.94113,0.20310],
                       [0.78563,0.93579,0.20336],
                       [0.79524,0.93025,0.20386],
                       [0.80473,0.92452,0.20459],
                       [0.81410,0.91861,0.20552],
                       [0.82333,0.91253,0.20663],
                       [0.83241,0.90627,0.20788],
                       [0.84133,0.89986,0.20926],
                       [0.85010,0.89328,0.21074],
                       [0.85868,0.88655,0.21230],
                       [0.86709,0.87968,0.21391],
                       [0.87530,0.87267,0.21555],
                       [0.88331,0.86553,0.21719],
                       [0.89112,0.85826,0.21880],
                       [0.89870,0.85087,0.22038],
                       [0.90605,0.84337,0.22188],
                       [0.91317,0.83576,0.22328],
                       [0.92004,0.82806,0.22456],
                       [0.92666,0.82025,0.22570],
                       [0.93301,0.81236,0.22667],
                       [0.93909,0.80439,0.22744],
                       [0.94489,0.79634,0.22800],
                       [0.95039,0.78823,0.22831],
                       [0.95560,0.78005,0.22836],
                       [0.96049,0.77181,0.22811],
                       [0.96507,0.76352,0.22754],
                       [0.96931,0.75519,0.22663],
                       [0.97323,0.74682,0.22536],
                       [0.97679,0.73842,0.22369],
                       [0.98000,0.73000,0.22161],
                       [0.98289,0.72140,0.21918],
                       [0.98549,0.71250,0.21650],
                       [0.98781,0.70330,0.21358],
                       [0.98986,0.69382,0.21043],
                       [0.99163,0.68408,0.20706],
                       [0.99314,0.67408,0.20348],
                       [0.99438,0.66386,0.19971],
                       [0.99535,0.65341,0.19577],
                       [0.99607,0.64277,0.19165],
                       [0.99654,0.63193,0.18738],
                       [0.99675,0.62093,0.18297],
                       [0.99672,0.60977,0.17842],
                       [0.99644,0.59846,0.17376],
                       [0.99593,0.58703,0.16899],
                       [0.99517,0.57549,0.16412],
                       [0.99419,0.56386,0.15918],
                       [0.99297,0.55214,0.15417],
                       [0.99153,0.54036,0.14910],
                       [0.98987,0.52854,0.14398],
                       [0.98799,0.51667,0.13883],
                       [0.98590,0.50479,0.13367],
                       [0.98360,0.49291,0.12849],
                       [0.98108,0.48104,0.12332],
                       [0.97837,0.46920,0.11817],
                       [0.97545,0.45740,0.11305],
                       [0.97234,0.44565,0.10797],
                       [0.96904,0.43399,0.10294],
                       [0.96555,0.42241,0.09798],
                       [0.96187,0.41093,0.09310],
                       [0.95801,0.39958,0.08831],
                       [0.95398,0.38836,0.08362],
                       [0.94977,0.37729,0.07905],
                       [0.94538,0.36638,0.07461],
                       [0.94084,0.35566,0.07031],
                       [0.93612,0.34513,0.06616],
                       [0.93125,0.33482,0.06218],
                       [0.92623,0.32473,0.05837],
                       [0.92105,0.31489,0.05475],
                       [0.91572,0.30530,0.05134],
                       [0.91024,0.29599,0.04814],
                       [0.90463,0.28696,0.04516],
                       [0.89888,0.27824,0.04243],
                       [0.89298,0.26981,0.03993],
                       [0.88691,0.26152,0.03753],
                       [0.88066,0.25334,0.03521],
                       [0.87422,0.24526,0.03297],
                       [0.86760,0.23730,0.03082],
                       [0.86079,0.22945,0.02875],
                       [0.85380,0.22170,0.02677],
                       [0.84662,0.21407,0.02487],
                       [0.83926,0.20654,0.02305],
                       [0.83172,0.19912,0.02131],
                       [0.82399,0.19182,0.01966],
                       [0.81608,0.18462,0.01809],
                       [0.80799,0.17753,0.01660],
                       [0.79971,0.17055,0.01520],
                       [0.79125,0.16368,0.01387],
                       [0.78260,0.15693,0.01264],
                       [0.77377,0.15028,0.01148],
                       [0.76476,0.14374,0.01041],
                       [0.75556,0.13731,0.00942],
                       [0.74617,0.13098,0.00851],
                       [0.73661,0.12477,0.00769],
                       [0.72686,0.11867,0.00695],
                       [0.71692,0.11268,0.00629],
                       [0.70680,0.10680,0.00571],
                       [0.69650,0.10102,0.00522],
                       [0.68602,0.09536,0.00481],
                       [0.67535,0.08980,0.00449],
                       [0.66449,0.08436,0.00424],
                       [0.65345,0.07902,0.00408],
                       [0.64223,0.07380,0.00401],
                       [0.63082,0.06868,0.00401],
                       [0.61923,0.06367,0.00410],
                       [0.60746,0.05878,0.00427],
                       [0.59550,0.05399,0.00453],
                       [0.58336,0.04931,0.00486],
                       [0.57103,0.04474,0.00529],
                       [0.55852,0.04028,0.00579],
                       [0.54583,0.03593,0.00638],
                       [0.53295,0.03169,0.00705],
                       [0.51989,0.02756,0.00780],
                       [0.50664,0.02354,0.00863],
                       [0.49321,0.01963,0.00955],
                       [0.47960,0.01583,0.01055]])




def RGBToPyCmap(rgbdata):
    nsteps = rgbdata.shape[0]
    stepaxis = np.linspace(0, 1, nsteps)

    rdata=[]; gdata=[]; bdata=[]
    for istep in range(nsteps):
        r = rgbdata[istep,0]
        g = rgbdata[istep,1]
        b = rgbdata[istep,2]
        rdata.append((stepaxis[istep], r, r))
        gdata.append((stepaxis[istep], g, g))
        bdata.append((stepaxis[istep], b, b))

    mpl_data = {'red':   rdata,
                 'green': gdata,
                 'blue':  bdata}

    return mpl_data


mpl_data = RGBToPyCmap(turbo_colormap_data)
plt.register_cmap(name='turbo', data=mpl_data, lut=turbo_colormap_data.shape[0])

mpl_data_r = RGBToPyCmap(turbo_colormap_data[::-1,:])
plt.register_cmap(name='turbo_r', data=mpl_data_r, lut=turbo_colormap_data.shape[0])
# FROM https://gist.github.com/FedeMiorelli/640bbc66b2038a14802729e609abfe89
import numpy as np
import matplotlib.pyplot as plt

turbo_colormap_data = np.array(
                       [[0.18995,0.07176,0.23217],
                       [0.19483,0.08339,0.26149],
                       [0.19956,0.09498,0.29024],
                       [0.20415,0.10652,0.31844],
                       [0.20860,0.11802,0.34607],
                       [0.21291,0.12947,0.37314],
                       [0.21708,0.14087,0.39964],
                       [0.22111,0.15223,0.42558],
                       [0.22500,0.16354,0.45096],
                       [0.22875,0.17481,0.47578],
                       [0.23236,0.18603,0.50004],
                       [0.23582,0.19720,0.52373],
                       [0.23915,0.20833,0.54686],
                       [0.24234,0.21941,0.56942],
                       [0.24539,0.23044,0.59142],
                       [0.24830,0.24143,0.61286],
                       [0.25107,0.25237,0.63374],
                       [0.25369,0.26327,0.65406],
                       [0.25618,0.27412,0.67381],
                       [0.25853,0.28492,0.69300],
                       [0.26074,0.29568,0.71162],
                       [0.26280,0.30639,0.72968],
                       [0.26473,0.31706,0.74718],
                       [0.26652,0.32768,0.76412],
                       [0.26816,0.33825,0.78050],
                       [0.26967,0.34878,0.79631],
                       [0.27103,0.35926,0.81156],
                       [0.27226,0.36970,0.82624],
                       [0.27334,0.38008,0.84037],
                       [0.27429,0.39043,0.85393],
                       [0.27509,0.40072,0.86692],
                       [0.27576,0.41097,0.87936],
                       [0.27628,0.42118,0.89123],
                       [0.27667,0.43134,0.90254],
                       [0.27691,0.44145,0.91328],
                       [0.27701,0.45152,0.92347],
                       [0.27698,0.46153,0.93309],
                       [0.27680,0.47151,0.94214],
                       [0.27648,0.48144,0.95064],
                       [0.27603,0.49132,0.95857],
                       [0.27543,0.50115,0.96594],
                       [0.27469,0.51094,0.97275],
                       [0.27381,0.52069,0.97899],
                       [0.27273,0.53040,0.98461],
                       [0.27106,0.54015,0.98930],
                       [0.26878,0.54995,0.99303],
                       [0.26592,0.55979,0.99583],
                       [0.26252,0.56967,0.99773],
                       [0.25862,0.57958,0.99876],
                       [0.25425,0.58950,0.99896],
                       [0.24946,0.59943,0.99835],
                       [0.24427,0.60937,0.99697],
                       [0.23874,0.61931,0.99485],
                       [0.23288,0.62923,0.99202],
                       [0.22676,0.63913,0.98851],
                       [0.22039,0.64901,0.98436],
                       [0.21382,0.65886,0.97959],
                       [0.20708,0.66866,0.97423],
                       [0.20021,0.67842,0.96833],
                       [0.19326,0.68812,0.96190],
                       [0.18625,0.69775,0.95498],
                       [0.17923,0.70732,0.94761],
                       [0.17223,0.71680,0.93981],
                       [0.16529,0.72620,0.93161],
                       [0.15844,0.73551,0.92305],
                       [0.15173,0.74472,0.91416],
                       [0.14519,0.75381,0.90496],
                       [0.13886,0.76279,0.89550],
                       [0.13278,0.77165,0.88580],
                       [0.12698,0.78037,0.87590],
                       [0.12151,0.78896,0.86581],
                       [0.11639,0.79740,0.85559],
                       [0.11167,0.80569,0.84525],
                       [0.10738,0.81381,0.83484],
                       [0.10357,0.82177,0.82437],
                       [0.10026,0.82955,0.81389],
                       [0.09750,0.83714,0.80342],
                       [0.09532,0.84455,0.79299],
                       [0.09377,0.85175,0.78264],
                       [0.09287,0.85875,0.77240],
                       [0.09267,0.86554,0.76230],
                       [0.09320,0.87211,0.75237],
                       [0.09451,0.87844,0.74265],
                       [0.09662,0.88454,0.73316],
                       [0.09958,0.89040,0.72393],
                       [0.10342,0.89600,0.71500],
                       [0.10815,0.90142,0.70599],
                       [0.11374,0.90673,0.69651],
                       [0.12014,0.91193,0.68660],
                       [0.12733,0.91701,0.67627],
                       [0.13526,0.92197,0.66556],
                       [0.14391,0.92680,0.65448],
                       [0.15323,0.93151,0.64308],
                       [0.16319,0.93609,0.63137],
                       [0.17377,0.94053,0.61938],
                       [0.18491,0.94484,0.60713],
                       [0.19659,0.94901,0.59466],
                       [0.20877,0.95304,0.58199],
                       [0.22142,0.95692,0.56914],
                       [0.23449,0.96065,0.55614],
                       [0.24797,0.96423,0.54303],
                       [0.26180,0.96765,0.52981],
                       [0.27597,0.97092,0.51653],
                       [0.29042,0.97403,0.50321],
                       [0.30513,0.97697,0.48987],
                       [0.32006,0.97974,0.47654],
                       [0.33517,0.98234,0.46325],
                       [0.35043,0.98477,0.45002],
                       [0.36581,0.98702,0.43688],
                       [0.38127,0.98909,0.42386],
                       [0.39678,0.99098,0.41098],
                       [0.41229,0.99268,0.39826],
                       [0.42778,0.99419,0.38575],
                       [0.44321,0.99551,0.37345],
                       [0.45854,0.99663,0.36140],
                       [0.47375,0.99755,0.34963],
                       [0.48879,0.99828,0.33816],
                       [0.50362,0.99879,0.32701],
                       [0.51822,0.99910,0.31622],
                       [0.53255,0.99919,0.30581],
                       [0.54658,0.99907,0.29581],
                       [0.56026,0.99873,0.28623],
                       [0.57357,0.99817,0.27712],
                       [0.58646,0.99739,0.26849],
                       [0.59891,0.99638,0.26038],
                       [0.61088,0.99514,0.25280],
                       [0.62233,0.99366,0.24579],
                       [0.63323,0.99195,0.23937],
                       [0.64362,0.98999,0.23356],
                       [0.65394,0.98775,0.22835],
                       [0.66428,0.98524,0.22370],
                       [0.67462,0.98246,0.21960],
                       [0.68494,0.97941,0.21602],
                       [0.69525,0.97610,0.21294],
                       [0.70553,0.97255,0.21032],
                       [0.71577,0.96875,0.20815],
                       [0.72596,0.96470,0.20640],
                       [0.73610,0.96043,0.20504],
                       [0.74617,0.95593,0.20406],
                       [0.75617,0.95121,0.20343],
                       [0.76608,0.94627,0.20311],
                       [0.77591,0.94113,0.20310],
                       [0.78563,0.93579,0.20336],
                       [0.79524,0.93025,0.20386],
                       [0.80473,0.92452,0.20459],
                       [0.81410,0.91861,0.20552],
                       [0.82333,0.91253,0.20663],
                       [0.83241,0.90627,0.20788],
                       [0.84133,0.89986,0.20926],
                       [0.85010,0.89328,0.21074],
                       [0.85868,0.88655,0.21230],
                       [0.86709,0.87968,0.21391],
                       [0.87530,0.87267,0.21555],
                       [0.88331,0.86553,0.21719],
                       [0.89112,0.85826,0.21880],
                       [0.89870,0.85087,0.22038],
                       [0.90605,0.84337,0.22188],
                       [0.91317,0.83576,0.22328],
                       [0.92004,0.82806,0.22456],
                       [0.92666,0.82025,0.22570],
                       [0.93301,0.81236,0.22667],
                       [0.93909,0.80439,0.22744],
                       [0.94489,0.79634,0.22800],
                       [0.95039,0.78823,0.22831],
                       [0.95560,0.78005,0.22836],
                       [0.96049,0.77181,0.22811],
                       [0.96507,0.76352,0.22754],
                       [0.96931,0.75519,0.22663],
                       [0.97323,0.74682,0.22536],
                       [0.97679,0.73842,0.22369],
                       [0.98000,0.73000,0.22161],
                       [0.98289,0.72140,0.21918],
                       [0.98549,0.71250,0.21650],
                       [0.98781,0.70330,0.21358],
                       [0.98986,0.69382,0.21043],
                       [0.99163,0.68408,0.20706],
                       [0.99314,0.67408,0.20348],
                       [0.99438,0.66386,0.19971],
                       [0.99535,0.65341,0.19577],
                       [0.99607,0.64277,0.19165],
                       [0.99654,0.63193,0.18738],
                       [0.99675,0.62093,0.18297],
                       [0.99672,0.60977,0.17842],
                       [0.99644,0.59846,0.17376],
                       [0.99593,0.58703,0.16899],
                       [0.99517,0.57549,0.16412],
                       [0.99419,0.56386,0.15918],
                       [0.99297,0.55214,0.15417],
                       [0.99153,0.54036,0.14910],
                       [0.98987,0.52854,0.14398],
                       [0.98799,0.51667,0.13883],
                       [0.98590,0.50479,0.13367],
                       [0.98360,0.49291,0.12849],
                       [0.98108,0.48104,0.12332],
                       [0.97837,0.46920,0.11817],
                       [0.97545,0.45740,0.11305],
                       [0.97234,0.44565,0.10797],
                       [0.96904,0.43399,0.10294],
                       [0.96555,0.42241,0.09798],
                       [0.96187,0.41093,0.09310],
                       [0.95801,0.39958,0.08831],
                       [0.95398,0.38836,0.08362],
                       [0.94977,0.37729,0.07905],
                       [0.94538,0.36638,0.07461],
                       [0.94084,0.35566,0.07031],
                       [0.93612,0.34513,0.06616],
                       [0.93125,0.33482,0.06218],
                       [0.92623,0.32473,0.05837],
                       [0.92105,0.31489,0.05475],
                       [0.91572,0.30530,0.05134],
                       [0.91024,0.29599,0.04814],
                       [0.90463,0.28696,0.04516],
                       [0.89888,0.27824,0.04243],
                       [0.89298,0.26981,0.03993],
                       [0.88691,0.26152,0.03753],
                       [0.88066,0.25334,0.03521],
                       [0.87422,0.24526,0.03297],
                       [0.86760,0.23730,0.03082],
                       [0.86079,0.22945,0.02875],
                       [0.85380,0.22170,0.02677],
                       [0.84662,0.21407,0.02487],
                       [0.83926,0.20654,0.02305],
                       [0.83172,0.19912,0.02131],
                       [0.82399,0.19182,0.01966],
                       [0.81608,0.18462,0.01809],
                       [0.80799,0.17753,0.01660],
                       [0.79971,0.17055,0.01520],
                       [0.79125,0.16368,0.01387],
                       [0.78260,0.15693,0.01264],
                       [0.77377,0.15028,0.01148],
                       [0.76476,0.14374,0.01041],
                       [0.75556,0.13731,0.00942],
                       [0.74617,0.13098,0.00851],
                       [0.73661,0.12477,0.00769],
                       [0.72686,0.11867,0.00695],
                       [0.71692,0.11268,0.00629],
                       [0.70680,0.10680,0.00571],
                       [0.69650,0.10102,0.00522],
                       [0.68602,0.09536,0.00481],
                       [0.67535,0.08980,0.00449],
                       [0.66449,0.08436,0.00424],
                       [0.65345,0.07902,0.00408],
                       [0.64223,0.07380,0.00401],
                       [0.63082,0.06868,0.00401],
                       [0.61923,0.06367,0.00410],
                       [0.60746,0.05878,0.00427],
                       [0.59550,0.05399,0.00453],
                       [0.58336,0.04931,0.00486],
                       [0.57103,0.04474,0.00529],
                       [0.55852,0.04028,0.00579],
                       [0.54583,0.03593,0.00638],
                       [0.53295,0.03169,0.00705],
                       [0.51989,0.02756,0.00780],
                       [0.50664,0.02354,0.00863],
                       [0.49321,0.01963,0.00955],
                       [0.47960,0.01583,0.01055]])




def RGBToPyCmap(rgbdata):
    nsteps = rgbdata.shape[0]
    stepaxis = np.linspace(0, 1, nsteps)

    rdata=[]; gdata=[]; bdata=[]
    for istep in range(nsteps):
        r = rgbdata[istep,0]
        g = rgbdata[istep,1]
        b = rgbdata[istep,2]
        rdata.append((stepaxis[istep], r, r))
        gdata.append((stepaxis[istep], g, g))
        bdata.append((stepaxis[istep], b, b))

    mpl_data = {'red':   rdata,
                 'green': gdata,
                 'blue':  bdata}

    return mpl_data


mpl_data = RGBToPyCmap(turbo_colormap_data)
plt.register_cmap(name='turbo', data=mpl_data, lut=turbo_colormap_data.shape[0])

mpl_data_r = RGBToPyCmap(turbo_colormap_data[::-1,:])
plt.register_cmap(name='turbo_r', data=mpl_data_r, lut=turbo_colormap_data.shape[0])

In [ ]:
fig, ax = plt.subplots(2, 2, figsize = (15, 15))
dcm.show(cmap='jet', title = 'jet', ax = ax[0, 0])
dcm.show(cmap='turbo', title = 'turbo', ax = ax[0, 1])
dcm.show(cmap='jet_r', title = 'jet_r', ax = ax[1, 0])
dcm.show(cmap='turbo_r', title = 'turbo_r', ax = ax[1, 1])

# Roman's Microscope Augmentation

**In the melanoma image, [Roman](https://www.kaggle.com/nroman) noticed that some images appeared as if they were taken through a microscope and he proposed a new type of augmentation where you augment all images such that they look like they were taken through a microscope. The link to his comment about this is [here](https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/159476) and his notebook is [here](https://www.kaggle.com/nroman/melanoma-pytorch-starter-efficientnet). We have the same problem in our dataset, so it might be useful to do that here:**

In [ ]:
#copy Roman's code from above discussion/notebook
import random
class Microscope:
    def __init__(self, p: float = 1):
        self.p = p

    def __call__(self, img):
        if random.random() < self.p:
            circle = cv2.circle((np.ones(img.shape) * 255).astype(np.uint8),
                        (img.shape[0]//2, img.shape[1]//2),
                        random.randint(img.shape[0]//2 - 3, img.shape[0]//2 + 15),
                        (0, 0, 0),
                        -1)

            mask = circle - 255
            img = np.multiply(img, mask)

        return img

In [ ]:
dcm_noscope = Path('../input/osic-pulmonary-fibrosis-progression/train/ID00009637202177434476278/18.dcm').dcmread()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 15))
ax[0].set_title('Before Microscope Augmentation')
ax[0].imshow(dcm_noscope.pixel_array, cmap = 'turbo')
ax[1].set_title('After Microscope Augmentation')
micro = Microscope()
ax[1].imshow(micro(dcm_noscope.pixel_array), cmap = 'turbo')
ax[0].axis('off')
ax[1].axis('off')
plt.show()

# For Next Time

**Now, the background of the above images should be black no matter which colormap, so there is some pixel reassignment to be done in the future...**

In [ ]:
#for future - gets PixelRepresentation and BitStored from dicom file
index = ['BitsStored','PixelRepresentation']
dicom_meta.pivot_table(values = ['img_mean','img_max','img_min','PatientID'], index = index,
                   aggfunc = {'img_mean':'mean','img_max':'max','img_min':'min','PatientID':'count'})